# *Import Library*

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# To get new datatypes and functions
from collections import Counter
from cycler import cycler

from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyRegressor

# *Dataset Upload*

In [3]:
columnname = pd.read_excel("features.xlsx")
column = columnname["degiskenismi"]

In [35]:
x_train = pd.read_excel("final_x_train.xlsx", names = column)
x_test = pd.read_excel("final_x_test.xlsx", names = column)
y_train = pd.read_excel("final_y_train.xlsx", names = ["activity"]) - 1
y_test = pd.read_excel("final_y_test.xlsx", names = ["activity"]) - 1

# *Dataset Overwiev*

In [6]:
x_train.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.29,-0.02,-0.18,-0.48,-0.45,-0.24,-0.53,-0.48,-0.25,-0.06,...,0.21,-0.47,-0.85,-0.34,0.94,-0.60,0.49,-0.59,0.10,0.31
1,0.17,0.01,-0.05,-0.61,-0.54,-0.36,-0.65,-0.55,-0.34,-0.54,...,0.12,-0.51,-0.86,0.67,-0.45,0.41,0.44,-0.58,0.09,0.32
2,0.39,-0.03,-0.01,-0.68,-0.47,-0.42,-0.71,-0.50,-0.42,-0.47,...,0.10,-0.63,-0.94,-0.14,0.42,0.55,0.09,-0.60,0.08,0.31
3,0.35,-0.02,-0.00,-0.65,-0.46,-0.36,-0.70,-0.50,-0.35,-0.47,...,0.49,-0.24,-0.55,-0.04,0.34,0.61,-0.42,-0.64,0.09,0.28
4,0.25,-0.00,-0.14,-0.62,-0.54,-0.21,-0.68,-0.55,-0.17,-0.43,...,0.17,-0.22,-0.58,0.13,-0.91,0.65,-0.76,-0.67,0.09,0.26


In [16]:
x_train.shape

(4251, 216)

In [7]:
y_train.head()

,activity
0,0
1,0
2,0
3,0
4,0


# *Group and count main names of columns* 

In [8]:
pd.DataFrame.from_dict(Counter([col.split('-')[0].split('(')[0] for col in x_train.columns]), 
                       orient='index').rename(columns={0:'count'}).sort_values('count', ascending=False)

,count
fBodyGyro,79
fBodyAcc,79
fBodyAccJerk,79
tBodyAcc,40
tBodyAccJerk,40
tBodyGyro,40
tBodyGyroJerk,40
tGravityAcc,40
fBodyBodyGyroJerkMag,13
fBodyBodyGyroMag,13


### *2 sensors, gyroscope (Gyro) and accelerometer(Acc) , were used in the data set.*

#### *In the Sensors.ipynb file, we modeled the data separately for each sensor and observed the success rates.What we're going to do here is decompose the variables in each sensor, apply principal component analysis, and reduce the variables in that sensor to a single component.*



# Gyroscope

In [12]:
x_train.shape

(4251, 561)

In [36]:
x_train_gyro = x_train.copy()

In [37]:
for var_name in x_train_gyro.columns:
    if 'Acc' in var_name:
        del x_train_gyro[var_name]

In [38]:
x_train_gyro.shape

(4251, 216)

In [39]:
x_train_gyro.head()

,tBodyGyro-mean()-X,tBodyGyro-mean()-Y,tBodyGyro-mean()-Z,tBodyGyro-std()-X,tBodyGyro-std()-Y,tBodyGyro-std()-Z,tBodyGyro-mad()-X,tBodyGyro-mad()-Y,tBodyGyro-mad()-Z,tBodyGyro-max()-X,...,fBodyBodyGyroJerkMag-entropy(),fBodyBodyGyroJerkMag-maxInds,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.36,-0.01,0.29,-0.34,-0.64,-0.41,-0.36,-0.65,-0.45,-0.28,...,0.30,-0.87,0.21,-0.47,-0.85,-0.60,0.49,-0.59,0.10,0.31
1,-0.21,0.04,-0.14,-0.29,-0.65,-0.43,-0.28,-0.67,-0.47,-0.42,...,0.28,-0.97,0.12,-0.51,-0.86,0.41,0.44,-0.58,0.09,0.32
2,-0.42,-0.04,-0.21,-0.38,-0.67,-0.49,-0.43,-0.69,-0.50,-0.42,...,0.27,-0.84,0.10,-0.63,-0.94,0.55,0.09,-0.60,0.08,0.31
3,-0.39,-0.12,-0.13,-0.55,-0.64,-0.49,-0.57,-0.64,-0.51,-0.69,...,0.18,-0.90,0.49,-0.24,-0.55,0.61,-0.42,-0.64,0.09,0.28
4,-0.10,-0.12,0.06,-0.44,-0.54,-0.48,-0.44,-0.56,-0.51,-0.56,...,0.38,-0.90,0.17,-0.22,-0.58,0.65,-0.76,-0.67,0.09,0.26


In [61]:
x_test_gyro = x_test.copy()

In [62]:
for var_name in x_test_gyro.columns:
    if 'Acc' in var_name:
        del x_test_gyro[var_name]

In [63]:
x_test_gyro.shape

(1491, 216)

In [67]:
x_test_gyro.head()

,tBodyGyro-mean()-X,tBodyGyro-mean()-Y,tBodyGyro-mean()-Z,tBodyGyro-std()-X,tBodyGyro-std()-Y,tBodyGyro-std()-Z,tBodyGyro-mad()-X,tBodyGyro-mad()-Y,tBodyGyro-mad()-Z,tBodyGyro-max()-X,...,fBodyBodyGyroJerkMag-entropy(),fBodyBodyGyroJerkMag-maxInds,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,-0.12,0.03,0.22,-0.82,-0.81,-0.77,-0.82,-0.83,-0.79,-0.74,...,-0.18,-1.00,0.06,0.75,0.64,0.13,0.41,-0.28,-0.13,-0.42
1,-0.07,0.02,0.12,-0.88,-0.81,-0.81,-0.87,-0.82,-0.82,-0.79,...,-0.13,-1.00,0.15,0.59,0.43,0.05,-0.15,-0.22,-0.13,-0.47
2,0.02,-0.18,-0.03,-0.92,-0.93,-0.90,-0.92,-0.94,-0.91,-0.78,...,-0.61,-0.97,0.57,-0.85,-0.98,0.14,-0.20,-0.10,-0.12,-0.56
3,-0.00,-0.15,0.03,-0.88,-0.92,-0.84,-0.87,-0.93,-0.83,-0.78,...,-0.60,-1.00,0.16,-0.65,-0.90,0.18,-0.19,-0.07,-0.13,-0.58
4,-0.05,-0.10,0.13,-0.90,-0.94,-0.85,-0.89,-0.95,-0.84,-0.83,...,-0.63,-1.00,0.13,-0.45,-0.79,0.10,-0.05,-0.10,-0.13,-0.56


## *Principal Component Analysis For Gyroscope*

In [17]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from pca import pca

In [70]:
pca = PCA(n_components=1)
pca_train_gyro_fit = pca.fit(x_train_gyro)

In [41]:
pca_gyro_x_train = pca.fit_transform(x_train_gyro)

In [42]:
pca_gyro_x_train = pd.DataFrame(pca_gyro_x_train)

In [43]:
pca_gyro_x_train.head(10)

,0
0,1.84
1,1.97
2,1.60
3,0.82
4,1.62
5,1.78
6,2.22
7,1.89
8,1.13
9,1.22


In [92]:
pca_gyro_x_train["Gyroscope"] = pca_gyro_x_train

In [95]:
pca_gyro_x_train.head()

,0,Gyroscope
0,1.84,1.84
1,1.97,1.97
2,1.60,1.60
3,0.82,0.82
4,1.62,1.62


In [109]:
pca_gyro_x_train.drop(pd.DataFrame(pca_gyro_x_train.iloc[:,0]),axis=1,inplace=True)

In [110]:
pca_gyro_x_train.head()

,Gyroscope
0,1.84
1,1.97
2,1.60
3,0.82
4,1.62


In [44]:
pca_gyro_x_train.shape

(4251, 1)

In [71]:
pca_test_gyro_fit = pca.fit(x_test_gyro)

In [72]:
pca_gyro_x_test = pca.fit_transform(x_test_gyro)

In [73]:
pca_gyro_x_test = pd.DataFrame(pca_gyro_x_test)

In [74]:
pca_gyro_x_test.head(10)

,0
0,-1.33
1,-1.42
2,-2.60
3,-2.64
4,-2.80
5,-2.76
6,-2.39
7,-2.33
8,-2.65
9,-2.69


In [75]:
pca_gyro_x_test.shape

(1491, 1)

In [111]:
pca_gyro_x_test["Gyroscope"] = pca_gyro_x_test

In [112]:
pca_gyro_x_test.drop(pd.DataFrame(pca_gyro_x_test.iloc[:,0]),axis=1,inplace=True)

In [114]:
pca_gyro_x_test.head()

,Gyroscope
0,-1.33
1,-1.42
2,-2.60
3,-2.64
4,-2.80


# Accelerometer

In [45]:
x_train.shape

(4251, 561)

In [50]:
x_train_acc = x_train.copy()

In [51]:
for var_name in x_train_acc.columns:
    if 'Gyro' in var_name:
        del x_train_acc[var_name]

In [59]:
x_train_acc.shape

(4251, 348)

In [60]:
x_train_acc.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyAccJerkMag-entropy(),fBodyBodyAccJerkMag-maxInds,fBodyBodyAccJerkMag-meanFreq(),fBodyBodyAccJerkMag-skewness(),fBodyBodyAccJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.29,-0.02,-0.18,-0.48,-0.45,-0.24,-0.53,-0.48,-0.25,-0.06,...,0.23,-0.90,-0.25,0.10,-0.21,-0.34,0.94,-0.59,0.10,0.31
1,0.17,0.01,-0.05,-0.61,-0.54,-0.36,-0.65,-0.55,-0.34,-0.54,...,0.05,-0.90,0.03,-0.08,-0.37,0.67,-0.45,-0.58,0.09,0.32
2,0.39,-0.03,-0.01,-0.68,-0.47,-0.42,-0.71,-0.50,-0.42,-0.47,...,0.09,-0.87,0.01,-0.28,-0.66,-0.14,0.42,-0.60,0.08,0.31
3,0.35,-0.02,-0.00,-0.65,-0.46,-0.36,-0.70,-0.50,-0.35,-0.47,...,0.09,-0.90,-0.07,-0.25,-0.58,-0.04,0.34,-0.64,0.09,0.28
4,0.25,-0.00,-0.14,-0.62,-0.54,-0.21,-0.68,-0.55,-0.17,-0.43,...,-0.05,-0.90,0.06,-0.02,-0.33,0.13,-0.91,-0.67,0.09,0.26


In [54]:
x_test_acc = x_test.copy()

In [55]:
for var_name in x_test_acc.columns:
    if 'Gyro' in var_name:
        del x_test_acc[var_name]

In [135]:
x_test_acc.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyAccJerkMag-entropy(),fBodyBodyAccJerkMag-maxInds,fBodyBodyAccJerkMag-meanFreq(),fBodyBodyAccJerkMag-skewness(),fBodyBodyAccJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.18,0.00,0.05,-0.60,-0.57,-0.49,-0.64,-0.57,-0.44,-0.48,...,-0.38,-1.00,0.03,0.48,0.25,-0.20,0.79,-0.28,-0.13,-0.42
1,-0.14,-0.05,0.11,-0.81,-0.76,-0.62,-0.83,-0.77,-0.61,-0.89,...,-0.28,-1.00,0.06,0.49,0.27,0.14,-0.69,-0.22,-0.13,-0.47
2,0.18,-0.00,-0.03,-0.83,-0.92,-0.68,-0.86,-0.92,-0.68,-0.86,...,-0.60,-0.97,-0.03,-0.55,-0.87,-0.15,-0.12,-0.10,-0.12,-0.56
3,0.37,-0.01,-0.17,-0.93,-0.89,-0.84,-0.93,-0.88,-0.82,-0.86,...,-0.90,-0.81,0.16,-0.74,-0.97,0.04,0.18,-0.07,-0.13,-0.58
4,0.30,-0.02,-0.12,-0.92,-0.89,-0.83,-0.92,-0.88,-0.82,-0.86,...,-0.94,-1.00,0.19,-0.61,-0.90,0.00,-0.08,-0.10,-0.13,-0.56


In [56]:
x_test_acc.shape

(1491, 348)

## *Principal Component Analysis For Accelerometer*

In [76]:
pca = PCA(n_components=1)
pca_acc_fit = pca.fit(x_train_acc)

In [77]:
pca_train_acc_fit = pca.fit(x_train_acc)

In [78]:
pca_acc_x_train = pca.fit_transform(x_train_acc)

In [79]:
pca_acc_x_train = pd.DataFrame(pca_acc_x_train)

In [81]:
pca_acc_x_train.head(10)

,0
0,2.74
1,1.60
2,1.44
3,1.52
4,1.46
5,2.40
6,3.36
7,3.15
8,2.40
9,1.77


In [82]:
pca_acc_x_train.shape

(4251, 1)

In [84]:
pca_test_acc_fit = pca.fit(x_test_acc)

In [85]:
pca_acc_x_test = pca.fit_transform(x_test_acc)

In [86]:
pca_acc_x_test = pd.DataFrame(pca_acc_x_test)

In [87]:
pca_acc_x_test.head(10)

,0
0,-0.80
1,-1.44
2,-2.83
3,-3.63
4,-4.03
5,-4.24
6,-3.77
7,-3.49
8,-3.60
9,-3.56


In [88]:
pca_acc_x_test.shape

(1491, 1)

In [115]:
pca_acc_x_train["Accelerometer"] = pca_acc_x_train

In [116]:
pca_acc_x_train.head()

,0,Accelerometer
0,2.74,2.74
1,1.60,1.60
2,1.44,1.44
3,1.52,1.52
4,1.46,1.46


In [117]:
pca_acc_x_train.drop(pd.DataFrame(pca_acc_x_train.iloc[:,0]),axis=1,inplace=True)

In [118]:
pca_acc_x_train.head()

,Accelerometer
0,2.74
1,1.60
2,1.44
3,1.52
4,1.46


In [119]:
pca_acc_x_test["Accelerometer"] = pca_acc_x_test

In [120]:
pca_acc_x_test.head()

,0,Accelerometer
0,-0.80,-0.80
1,-1.44,-1.44
2,-2.83,-2.83
3,-3.63,-3.63
4,-4.03,-4.03


In [121]:
pca_acc_x_test.drop(pd.DataFrame(pca_acc_x_test.iloc[:,0]),axis=1,inplace=True)

In [122]:
pca_acc_x_test.head()

,Accelerometer
0,-0.80
1,-1.44
2,-2.83
3,-3.63
4,-4.03


### *Let's combine the acc and gyro variables in the train set*

In [127]:
x_train_pca = pd.concat([pca_gyro_x_train, pca_acc_x_train],axis=1)

In [128]:
x_train_pca.head()

,Gyroscope,Accelerometer
0,1.84,2.74
1,1.97,1.60
2,1.60,1.44
3,0.82,1.52
4,1.62,1.46


In [129]:
x_train_pca.shape

(4251, 2)

### *Let's combine the acc and gyro variables in the test set*

In [130]:
x_test_pca = pd.concat([pca_gyro_x_test, pca_acc_x_test],axis=1)

In [131]:
x_test_pca.head()

,Gyroscope,Accelerometer
0,-1.33,-0.80
1,-1.42,-1.44
2,-2.60,-2.83
3,-2.64,-3.63
4,-2.80,-4.03


In [133]:
x_test_pca.shape

(1491, 2)

# *Setting up models*

In [134]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(x_train_pca, x_test_pca, y_train, y_test)
models

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:13<00:00,  2.16it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
XGBClassifier,0.48,0.48,None,0.48,2.10
KNeighborsClassifier,0.48,0.48,None,0.48,0.08
LogisticRegression,0.48,0.47,None,0.44,0.16
LGBMClassifier,0.47,0.47,None,0.47,0.87
CalibratedClassifierCV,0.48,0.47,None,0.42,1.40
SVC,0.47,0.46,None,0.43,1.51
RandomForestClassifier,0.46,0.46,None,0.46,0.81
ExtraTreesClassifier,0.46,0.46,None,0.46,0.59
LinearDiscriminantAnalysis,0.47,0.46,None,0.42,0.03
